# 矩阵分解

In [1]:
import numpy as np
import scipy
from scipy import linalg
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display,Latex,Math
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
sh = InteractiveShell.instance()

def number_to_str(n,cut=5):
    ns=str(n)
    format_='{0:.'+str(cut)+'f}'
    if 'e' in ns or ('.' in ns and len(ns)>cut+1):
        return format_.format(n)
    else:
        return str(n)

def matrix_to_latex(mat,style='bmatrix'):
    if type(mat)==np.matrixlib.defmatrix.matrix:
        mat=mat.A
    head=r'\begin{'+style+'}'
    tail=r'\end{'+style+'}'
    if len(mat.shape)==1:
        body=r'\\'.join([str(el) for el in mat])
        return head+body+tail
    elif len(mat.shape)==2:
        lines=[]
        for row in mat:
            lines.append('&'.join([number_to_str(el)  for el in row])+r'\\')
        s=head+' '.join(lines)+tail
        return s
    return None

sh.display_formatter.formatters['text/latex'].type_printers[np.ndarray]=matrix_to_latex

def show_decomposition(*args):
    latex=''
    for arg in args:
        if type(arg)==str:
            latex+=arg
        else:
            latex+=matrix_to_latex(arg)
    latex='$'+latex+'$'
    display(Math(latex))


## 三角分解

三角分解就是把矩阵分解成上下三角的乘积形式，这样就可以利用回代法直接得到解。

### 回代法

考虑形式为

$$
\begin{bmatrix}
 1 & 2 & 3 \\
 0 & 4 & 5 \\
 0 & 0 & 6 \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\end{bmatrix}
=
\begin{bmatrix}
7 \\
8 \\
9 \\
\end{bmatrix}\\
$$
的方程，显然可以从右下网上机械的求解

In [2]:
def subsup(A,b):
    size=A.shape[1]
    x=np.zeros(size)
    for i in range(size-1,-1,-1):
        x[i]=(b[i]-sum([x[j]*A[i][j] for j in range(i+1,size)]))/A[i][i]   
    return x



In [3]:
A=np.array([[1,2,3],[0,4,5],[0,0,6]])
b=np.array([8,9,10])
subsup(A,b)

array([ 2.66666667,  0.16666667,  1.66666667])

In [4]:
np.linalg.solve(A,b)

array([ 2.66666667,  0.16666667,  1.66666667])

In [5]:
A=np.array([1,2,3,0,4,5,0,0,6]).reshape(3,3)
b=np.array([7,8,9])

x=subsup(A,b)
x

array([ 2.25 ,  0.125,  1.5  ])

In [6]:
show_decomposition(A,x,'=',A.dot(x))

<IPython.core.display.Math object>

下三角的情况类似
$$
\begin{bmatrix}
 1 & 0 & 0 \\
 2 & 3 & 0 \\
 4 & 5 & 6 \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\end{bmatrix}
=
\begin{bmatrix}
7 \\
8 \\
9 \\
\end{bmatrix}\\
$$


In [7]:
def subsdown(A,b):
    size=A.shape[1]
    x=np.zeros(size)
    for i in range(size):
        x[i]=(b[i]-sum([A[i][j]*x[j] for j in range(i)]))/A[i][i]
    return x

In [8]:
A=np.array([1,0,0,2,3,0,4,5,6]).reshape(3,3)
b=np.array([7,8,9])
x=subsdown(A,b)
x

array([ 7. , -2. , -1.5])

In [9]:
show_decomposition(A,x,'=',A.dot(x))

<IPython.core.display.Math object>

从而我们也就见识了上下三角矩阵的回代如何容易解决这种方成求解问题。那么显然，对于一般的

$$
Ax=b
$$

问题，我们如果能将$A$分解成下三角矩阵$L$与上三角矩阵$R$。则我们换元来利用回代法解决问题。

$$
LRx=b \\
Rx=y \\
Ly=b \\
$$

于是可以先利用下三角矩阵回代法求出$y$，再由$Rx=y$求出$x$。这种方法不知为何有种似曾相识的感觉...

### LR分解

LR分解还可以要求L的对角线上元全为1.

$$
\begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn} \\
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & \cdots & 0 \\
l_{21} & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
l_{n1} & l_{n2} & \cdots & 1 \\
\end{bmatrix}
\begin{bmatrix}
r_{11} & r_{12} & \cdots & r_{1n} \\
0 & r_{22} & \cdots & r_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & r_{nn} \\
\end{bmatrix}
$$

In [10]:
def lr(A):
    size=A.shape[0]
    L=np.diag(np.ones(size))
    R=np.zeros(A.shape)
    for t in range(size):
        R[0][t]=A[0][t]
    for l in range(1,size):
        for i in range(l):
            L[l][i]=(A[l][i]-sum([L[l][jj]*R[jj][i] for jj in range(i) ]))/R[i][i]
        for j in range(l,size):
            R[l][j]=A[l][j]-sum([L[l][jj]*R[jj][j] for jj in range(l)])
    return L,R


            

In [11]:
A=np.array([2,3,0,0,1,0,4,6,0]).reshape(3,3)

A

array([[2, 3, 0],
       [0, 1, 0],
       [4, 6, 0]])

In [12]:
L,R=lr(A)

show_decomposition(L.dot(R),'=',L,R)

<IPython.core.display.Math object>

如此，根据上面的换元思想，就可以定义出一个基于lr分解的方程组的解法，当然在这里并没有显示出比高斯消元法任何的优势。

In [13]:
def solve(A,b):
    L,R=lr(A)
    y=subsdown(L,b)
    x=subsup(R,y)
    return x

In [14]:
A=(np.arange(1,10)**2).reshape(3,3)
A

array([[ 1,  4,  9],
       [16, 25, 36],
       [49, 64, 81]])

In [15]:
b=np.arange(1,4)
b

array([1, 2, 3])

In [16]:
np.linalg.solve(A,b)

array([ 0.08333333, -0.33333333,  0.25      ])

In [17]:
solve(A,b)

array([ 0.08333333, -0.33333333,  0.25      ])

In [18]:
L,R=lr(A)

show_decomposition(L.dot(R),'=',L,R)

<IPython.core.display.Math object>

In [19]:
solve(A,b) # LR decomposition way

array([ 0.08333333, -0.33333333,  0.25      ])

### $LDR^*$分解

两个三角矩阵左边的对角线上被规范为1，一个没规范，不对称不能忍，所以导出$LDR^*$分解，通过从$R$中抽出一个对角矩阵来把对角线也归1化使其对称。

In [20]:
def ldrstar(A):
    size=A.shape[0]
    L,R=lr(A)
    #Rstar=np.zeros([size,size])
    Rstar=np.identity(size)
    D=np.diag(R.diagonal())
    for i in range(size-1):
        for j in range(i+1,size):
            Rstar[i][j]=R[i][j]/D[i][i]
    return L,D,Rstar

A=np.array([2,3,0,0,1,0,4,6,0]).reshape(3,3)
A
    

array([[2, 3, 0],
       [0, 1, 0],
       [4, 6, 0]])

In [21]:
L,D,Rstar=ldrstar(A)
show_decomposition(L.dot(D).dot(Rstar),'=',L,D,Rstar)

<IPython.core.display.Math object>

### $L^*R^*$ 分解

$L^*R^*$分解就是把$L$的全1对角线性质移到了$R$上。显然，既然有了$LDR^*$式，只要令$LD=L^*$即可

In [22]:
def lstarrstar(A):
    L,D,Rstar=ldrstar(A)
    Lstar=np.zeros(L.shape)
    for i in range(L.shape[0]):
        for j in range(L.shape[1]):
            Lstar[i][j]=L[i][j]*D[j][j]
    return Lstar,Rstar

Lstar,Rstar=lstarrstar(A)
show_decomposition(Lstar.dot(Rstar),'=',Lstar,Rstar)

<IPython.core.display.Math object>

### 存在唯一性

theroem I

A的LR分解存在且唯一，当且仅当其直到n-1顺序主子式不等于0.

## Cholesky 分解

对于对称正定矩阵，具有更强的三角分解形式。其可以被分解为一对互为转置上/下三角矩阵的乘积。

$$
A=TT^{'}
$$

In [23]:
def cholesky(A):
    size=A.shape[0]
    T=np.zeros(A.shape)
    for i in range(size):
        T[i][i]=np.sqrt(A[i][i]-sum([T[i][t]**2 for t in range(i)]))
        for j in range(i+1,size):
            T[j][i]=(A[i][j]-sum([T[i][t]*T[j][t] for t in range(i)]))/T[i][i]
    return T

In [24]:
A=np.array([4,6,10,6,58,29,10,29,38]).reshape(3,3)
A

array([[ 4,  6, 10],
       [ 6, 58, 29],
       [10, 29, 38]])

In [25]:
T=cholesky(A)
T

array([[ 2.,  0.,  0.],
       [ 3.,  7.,  0.],
       [ 5.,  2.,  3.]])

In [26]:
T.dot(np.transpose(T))

array([[  4.,   6.,  10.],
       [  6.,  58.,  29.],
       [ 10.,  29.,  38.]])

### QR分解

三角分解将矩阵分解为一些三角矩阵和数量阵的乘积。QR分解则将其分解为一个正交阵与三角阵的乘积。

$$
A=QR
$$
其中Q是正交矩阵,R是上三角矩阵。

#### Householder 变换

这个变换由一个超平面确定，将点映射到与相对超平面另一边对称的点上。从计算的角度来看，超平面可以由与它正交的一个向量$u$所确定。
这个变换的矩阵可以写成
$$
H=1-uu^T
$$
其中u是列向量，或者说$n \times 1$矩阵。

#### 基于Householder变换的QR分解

设想有一系列正交变换，其每次将原矩阵一个列像高斯消元法一样变成某个元以下全为0.则全部变换完之后，所有作用过的正交矩阵复合成一个正交矩阵，剩下的矩阵正是一个（正交）消元法剩下的上三角矩阵，这就做成了QR分解。

那么问题是，行初等变换的矩阵一般不是正交矩阵，我们必须构造出一种消元的正交矩阵方法，既然它保持长度，一个显然的想法就是它同时包含直接消元又包含将剩下的元恢复为原长度两步。

换个思路，假如我们限定为考虑第一步，将第一列变为一个除第一个元为列长度外其他元皆为0的向量，这可以通过找到一个Householder变换
对应的超平面（即原向量与变换后的轴向量“中间”那个平面），也可以转化为找到这个平面的那个正交向量。通过几何直观我们发现，原向量$x$与目标向量$\lVert x \rVert e_1$所需要的正交向量就是它们的差$x-\lVert x \rVert e_1$.我们令这个向量为$u$,同时将其标准化，以便利用其定义出Householder变换矩阵，

$$
\begin{align}
& u=x-\lVert x \rVert e_1 \\
& v=\frac{u}{\lVert u \rVert} \\
& Q=I-2vv^T
\end{align}
$$

定义出的变换矩阵$Q$具有性质

$$
Qx=
\begin{bmatrix}
\lVert x \rVert \\
0 \\
\vdots \\
0 \\
\end{bmatrix}
$$
这正是我们想要的

In [27]:
A=np.array([12,-51,4,6,167,-68,-4,24,-41]).reshape(3,3)
A

array([[ 12, -51,   4],
       [  6, 167, -68],
       [ -4,  24, -41]])

In [28]:
x=A[:,0]
x

array([12,  6, -4])

In [29]:
alpha=np.sqrt(x.dot(x))
alpha

14.0

In [30]:
e1=np.zeros(x.shape[0])
e1[0]=1
u=x-alpha*e1
u

array([-2.,  6., -4.])

In [31]:
v=u/np.sqrt(u.dot(u))
v

array([-0.26726124,  0.80178373, -0.53452248])

In [32]:
Q=np.identity(v.shape[0])-2*np.outer(v,v)
Q

array([[ 0.85714286,  0.42857143, -0.28571429],
       [ 0.42857143, -0.28571429,  0.85714286],
       [-0.28571429,  0.85714286,  0.42857143]])

In [33]:
Q.dot(x)

array([  1.40000000e+01,  -4.44089210e-16,   4.44089210e-16])

In [34]:
def get_Q1(x):
    size=x.shape[0]
    norm_x=np.sqrt(np.dot(x,x))
    e1=np.zeros(size)
    e1[0]=1
    u=x-norm_x*e1
    norm_u=np.sqrt(np.dot(u,u))
    v=u/norm_u
    Q=np.identity(size)-2*np.outer(v,np.transpose(v))
    return Q

get_Q1(np.array([12,6,-4]))

array([[ 0.85714286,  0.42857143, -0.28571429],
       [ 0.42857143, -0.28571429,  0.85714286],
       [-0.28571429,  0.85714286,  0.42857143]])

In [39]:
from functools import reduce

def qr(A):
    size=A.shape[1]
    QList=[]
    for i in range(size):
        x=A[i:,i]
        Q=get_Q1(x)
        Qn=np.identity(A.shape[0])
        #print(Qn.shape,Q.shape)
        Qn[i:,i:]=Q
        A=np.dot(Qn,A)
        QList.append(Qn)
    Q=reduce(lambda x,y:x.dot(y),QList)
    R=A
    return Q,R
    

In [57]:
A=np.array([12,-51,4,6,167,-68,-4,24,-41]).reshape(3,3)

A

array([[ 12, -51,   4],
       [  6, 167, -68],
       [ -4,  24, -41]])

In [58]:
Q,R=qr(A)

show_decomposition(Q.dot(R),'=',Q,R)

<IPython.core.display.Math object>

In [42]:
A=np.arange(1,16).reshape(5,3)

Q,R=qr(A)

show_decomposition(Q.dot(R),'=',Q,R)

<IPython.core.display.Math object>

In [43]:
#thin mode
show_decomposition(Q.dot(R),'=',Q[:,:-2],R[:-2,:])

<IPython.core.display.Math object>

## QR分解与最小二乘法

最小二乘问题是

$$
\min_x \lVert b-Ax \rVert_2
$$

设A具有QR分解式
$$
A=QR=Q_1R_1
$$
其中$Q_1$$R_1$是$Q$,$R$的短版本（即去掉了全0那块及相关地方的剩下来的矩阵）

$$
\lVert b-Ax \rVert_2=\left \lVert  \begin{bmatrix} Q_1^Tb \\ Q_2^Tb \end{bmatrix} - \begin{bmatrix} Rx \\ 0 \\ \end{bmatrix} \right \rVert_2^2 
=
\lVert Q_1^Tb -Rx \rVert_2^2 + \lVert Q_2^Tb \rVert_2^2
$$

可以注意到莫名其妙的将误差分解成了可以降成0和完全不能变的两部分，所以只要单纯令左边等于0就能得到最小二乘解（代数里这一套其实经常出现）。
得到了

$$
x=R^{-1}Q_1^Tb \\
$$

其中R因为是上三角矩阵，Q_1是正交矩阵，都很容易求逆（这里直接写成了取转置。正交矩阵转置为逆这一性质是在推导中追求它的主要原因）

In [44]:
def R_I(A):
    #上三角矩阵R -> R^I
    A=A.copy()
    size=A.shape[0]
    I=np.identity(size)
    for i in range(size-1,-1,-1):
        I[i,:]=I[i,:]/A[i][i]
        A[i,:]=A[i,:]/A[i][i]
        for j in range(i):
            c=-A[j][i]
            A[j,:]=A[j,:]+c*A[i,:]
            I[j,:]=I[j,:]+c*I[i,:]
    return I
    
def qr_solve(A,b):
    Q,R=qr(A)
    attrs=A.shape[1]
    Q1=Q[:,:attrs]
    R1=R[:attrs,:]
    RI=R_I(R1)
    return RI.dot(np.transpose(Q1)).dot(b)


In [45]:
R=np.array([1.0,2.0,3.0,0.0,4.0,5.0,0.0,0.0,6.0]).reshape(3,3)
R

array([[ 1.,  2.,  3.],
       [ 0.,  4.,  5.],
       [ 0.,  0.,  6.]])

In [46]:
RI=R_I(R)

RI.dot(R)

array([[  1.00000000e+00,   0.00000000e+00,  -2.22044605e-16],
       [  0.00000000e+00,   1.00000000e+00,   1.11022302e-16],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00]])

In [47]:
A=np.sqrt(np.linspace(1,100,30).reshape(10,3))
b=np.linspace(1,10,10)

qr_solve(A,b)

array([-10.71889453,  30.52391485, -18.71361375])

In [48]:
np.linalg.lstsq(A,b)

(array([-10.71889453,  30.52391485, -18.71361375]),
 array([ 1.45354127]),
 3,
 array([ 38.8997761 ,   1.34317996,   0.05733151]))

很好，取得了和`np.linalg.lstsq`一样的结果

In [52]:
A=np.array([[1,2,3],[4,5,6],[7,8,9]])**2
Q,R=qr(A)
Q

array([[ 0.01939646,  0.56467104,  0.82508811],
       [ 0.31034339,  0.7810805 , -0.54184891],
       [ 0.95042662, -0.26657059,  0.16009172]])

In [53]:
R

array([[  5.15557950e+01,   6.86634742e+01,   8.83314863e+01],
       [  5.74235325e-16,   4.72517870e+00,   1.16087193e+01],
       [  7.69785590e-15,   1.05141559e-15,   8.86661855e-01]])

In [55]:
get_Q1(np.array([1,2,3]))

array([[ 0.26726124,  0.53452248,  0.80178373],
       [ 0.53452248,  0.61007346, -0.5848898 ],
       [ 0.80178373, -0.5848898 ,  0.12266529]])

In [61]:
import scipy.stats as stat


In [62]:
A=stat.norm(0,1).rvs(size=300).reshape(100,3)

In [64]:
coef=np.array([1,2,3])


In [68]:
e=stat.norm(0,1).rvs(size=100)

In [69]:
ey=A.dot(coef)
y=ey+e

In [70]:
qr_solve(A,y)

array([ 0.88970663,  2.04661238,  2.8226361 ])

In [71]:
np.linalg.lstsq(A,y)

(array([ 0.88970663,  2.04661238,  2.8226361 ]),
 array([ 91.31112813]),
 3,
 array([ 11.70797844,  10.0783936 ,   9.59738953]))

In [72]:
qr_solve(A,ey)

array([ 1.,  2.,  3.])